In [ ]:
#origin code

# -*- coding: utf-8 -*-
"""
Created on Thu Sep 29 01:42:29 2022

@author: ASUS

исходный файл: эксель со столбцами: 
    PDBid	chain	start*	end*
*-по PDB файлу

аут файл:


Описание функций:
    def Start - указываем путь к файлу xlsx, на выходе считанный датафрейм. [Вход: путь к экселю. Выход: считанный датафрейм]
    def Change_df - (!!! меняет исходный датафрейм) преобразует датафрейм к виду PDBid_chain PDBid chain start end length [Вход: датафрейм. Выход: преобразованный датафрейм. Сохраняет: эксель с преобразованным датафреймом]
    def Hist_maker - строит гистограмму по длинам подборки [Вход: подборка с длинами. Сохраняет: картинку гистограмму распределения длин молекул подборки]
    def Alpha_helix - добывает координаты альфа-спиралей [Вход: подборка наша. Вызывает: list_struc_maker (выход: датафрейм со спиралями их координатами)]
    def list_struc_maker - создаёт список длиной скоько а.о. и заполняет 1 там где есть структура [Вход: датафрейм со спиралями Helixes, длина белка l, начало белка start_domain. Вызывается в Alpha_helix]
    
"""

import numpy as np
import pandas as pd
import time
#import seaborn as sns
#import matplotlib.pyplot as plt
#from tempfile import gettempdir
#import biotite
#import biotite.structure as struc
#import biotite.structure.io.mmtf as mmtf
#import biotite.structure.io.pdb as pdb
import biotite.structure.io.pdbx as pdbx
#import biotite.sequence as seq
#import biotite.sequence.graphics as graphics
import biotite.database.rcsb as rcsb
#import biotite.application.dssp as dssp

#_______________________________________________________

def Start ():
    #-----Формируем первый датафрейм {PDBid chain start end}
    #print('Введите путь к файлу excel с его наименованием и .xlsx')
    #way = str(input())
    #'D:/Оля/курсовая/1_Обработка/1 Работа/проверяю расширенную обработку/pdbID.txt'
    path = 'book.xlsx'
    print("Вы ввели", path) 
    Podborka = pd.read_excel(path) #читвем датафрейм в переменную Podborka
    #print("Original Podborka\n", Podborka.head())
    #print('Типы:', Podborka.dtypes)
    #print('Колонки', Podborka.columns)
    #Podborka['PDBid']=str(Podborka['PDBid'])#преобразуем в строку
    #Podborka['chain']=Podborka['chain'].astype(str)#преобразуем в строку
    #Podborka['PDBid'].astype(str)
    #print('Типы:', Podborka.dtypes)
    print('--------------------')
    return Podborka
    
def Change_df (Podborka):
    #-----Модифицируем датафрейм, добавляя length и PDBid_chain {PDBid_chain PDBid chain start end length}
    Podborka.insert(4, 'length', Podborka['end']-Podborka['start']+1) #считаем длину в а.о. каждого домена/цепи
    Podborka.insert(0, 'PDBid_Chain', Podborka['PDBid']+'_'+Podborka['chain'])#добавим идетификатор PDBid_Chain
    #print("Changed Podborka\n", Podborka.head())
    #Выведем саммари по этому датафрейму
    print('Саммари\n', Podborka.describe())  
    #Podborka.describe().to_csv("Podborka_describe.csv")
    Podborka.describe().to_excel("Podborka_describe.xlsx")#запишем себе данные дискрайба
    #и на всякий случай сохраним датафрейм
    Podborka.to_excel("Podborka.xlsx")
    #Заготовки для результатов по стурктурам
    Podborka["Alpha-helixes"] = np.nan
    Podborka["Beta-sheets"] = np.nan
    Podborka["ThreeTen-helixes"] = np.nan
    Podborka["Count_AlphaH"] = np.nan
    Podborka["Count_BetaS"] = np.nan
    Podborka["Count_ThreeTenH"] = np.nan
    return Podborka
    print('--------------------')

def Hist_maker(Podborka):
    #Сразу распечатаем гистограмму (распределение длин белков подборки)
    ax = Podborka.hist(column='length', bins=25, grid=True, figsize=(12,8), color='royalblue', rwidth=0.9)  
    ax = ax[0]
    for x in ax:
        # Remove title
        x.set_title("Распределение длин белков подборки", size=20)
        # Set x-axis label
        x.set_xlabel("Длина, а.о.", labelpad=20, weight='bold', size=16)
        # Set y-axis label
        x.set_ylabel("Количество, структур", labelpad=20, weight='bold', size=16)
    fig = ax[0].get_figure()
    fig.savefig('Length distribution.jpg')#сохраним гистограмму
    print('--------------------')

def Alpha_helix(Podborka):
    g = 0
    for i, row in Podborka.iterrows():
        #input()
        #print("ALPHA")
        #print(f"Index: {i}")
        #print(f"{row}\n")
        PDBid = Podborka.iloc[i, 1]#выясняем актуальный PDBid 
        l = Podborka.iloc[i, 5]#выясняем актуальную длину
        chain_tut = Podborka.iloc[i, 2]#выясняем актуальную цепь
        start_domain = int(Podborka.iloc[i, 3])#выясняем актуальную координату начала домена/белка
        stop_domain = int(Podborka.iloc[i, 4])#выясняем актуальную координату конца домена/белка
        print("Данные: ", PDBid, l, chain_tut, start_domain, stop_domain, "g=", g)
        g=g+1
        #читаем PDBx из инета
        pdbx_file = pdbx.PDBxFile.read(rcsb.fetch(PDBid, "pdbx"))
        #Получаем данные из файла вообще есть они там или нет
        P=pdbx_file [PDBid, "struct_conf"]#инфа о листах
        #print(pdbx_file["struct_sheet_order"])
        #input()
        #тут проверяем есть ли вообще данные о листах
        if P is not None:
            Id_hel=pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_id"]#id спиралей внутри файла в список
            #print(Id_hel)
            Beg_str = pdbx_file [PDBid, "struct_conf"]["beg_label_seq_id"]#начала спиралей а.о. для PDB обычного
            #Beg_str = pdbx_file [PDBid, "struct_conf"]["beg_auth_seq_id"]#начала спиралей а.о. для CATH
            #print("Beg_str=", Beg_str, "type=", type(Beg_str))
            if isinstance(Beg_str, str):
                Beg_str=np.array([int(Beg_str)]) 
            else:
                Beg_str = Beg_str.astype("int")
            #Beg_str = Beg_str.astype("int")
            #print(Beg_str)
            End_str = pdbx_file [PDBid, "struct_conf"]["end_label_seq_id"]#концы спиралей а.о.
            #End_str = pdbx_file [PDBid, "struct_conf"]["end_auth_seq_id"]#концы спиралей а.о.
            if isinstance(End_str, str):
                End_str=np.array([int(End_str)])
            else:
                End_str = End_str.astype("int")
            Class_hel = pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_class"]#классы спирали
            Length_hel = pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_length"]#длины спирали
            #Chain_hel = pdbx_file [PDBid, "struct_conf"]["beg_label_asym_id"]#цепи белка по буквам, каким принадлежат эти спирали
            Chain_hel = pdbx_file [PDBid, "struct_conf"]["beg_auth_asym_id"]#цепи белка по буквам, каким принадлежат эти спирали
            #print(Chain_hel)
            #ещё одна проверка если вдруг будет тупняк
            if Id_hel is not None:
                #сформируем локальный датафрейм альфа-спиралей, если эти альфа-спирали есть
                Helixes = pd.DataFrame({'Id_hel' : Id_hel,'Beg_str' : Beg_str, 'End_str': End_str, 'Class_hel': Class_hel, 'Length_hel': Length_hel, 'Chain_hel': Chain_hel})
                #print('Helixes\n', Helixes)
                #print('type', type(Helixes))
                #Удалим ненужные нам колонки (по названию цепи смотрим)
                Helixes = Helixes[Helixes.Class_hel == "1"]#тип спирали == альфа
                Helixes = Helixes[Helixes.Chain_hel == chain_tut]#цепь нужная по Podborka
                Helixes = Helixes[Helixes.Beg_str >= start_domain] #входит в домен
                Helixes = Helixes[Helixes.End_str <= stop_domain] #входит в домен
                Helixes.index = np.arange(len(Helixes))#обновляем индексы (чтобы стали от 0 до ..., а не как были в Helixes до того как удалим ненужные нам цепи)
                #print(" Helixes.index",  Helixes.index)
                #print("Helix которые подходят нам\n", Helixes)
                Podborka.loc[i, "Count_AlphaH"] = Helixes.shape[0]
                #input()
                #если пусто по листам (их нет нужных), то тоже будем писать NaN или нули. тут повторная штука такая т.к. листов может не быть изначально (выше) или их не будет лишь в нужных нам цепях белка (тут)
                isempty = Helixes.empty#если нет альфа-спиралей, то мы можем выбрать как заполнить данные
                if isempty:
                    print("В белке ", PDBid, "альфа-спиралей не обнаружено")
                    Res=[0]*100
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "Alpha-helixes"] = vse100
                    #print("Podborka", Podborka)
                else:
                    Res=[]
                    Res=list_struc_maker(Helixes, l, start_domain)#Получаем 100% данные структур
                    #print("Res", Res)
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "Alpha-helixes"] = vse100
                #print("Podborka", Podborka)
            else:
                print("Какие-то проблемы с Id_hel")
        else:
            print("В белке ", PDBid, "альфа-спиралей не обнаружено")
            Podborka.loc[i, "Count_AlphaH"] = 0
            Res=[0]*100
            vse100 = (";".join(map(str,Res)))
            Podborka.loc[i, "Alpha-helixes"] = vse100
            #print("Podborka", Podborka
        print('--------------------')
    
def ThreeTen_helix(Podborka):
    g=0
    for i, row in Podborka.iterrows():
        #input()
        #print("ALPHA")
        #print(f"Index: {i}")
        #print(f"{row}\n")
        PDBid = Podborka.iloc[i, 1]#выясняем актуальный PDBid 
        l = Podborka.iloc[i, 5]#выясняем актуальную длину
        chain_tut = Podborka.iloc[i, 2]#выясняем актуальную цепь
        start_domain = int(Podborka.iloc[i, 3])#выясняем актуальную координату начала домена/белка
        stop_domain = int(Podborka.iloc[i, 4])#выясняем актуальную координату конца домена/белка
        print("Данные: ", PDBid, l, chain_tut, start_domain, stop_domain, "g=", g)
        g=g+1
        #читаем PDBx из инета
        pdbx_file = pdbx.PDBxFile.read(rcsb.fetch(PDBid, "pdbx"))
        #Получаем данные из файла вообще есть они там или нет
        P=pdbx_file [PDBid, "struct_conf"]#инфа о листах
        #print(pdbx_file["struct_sheet_order"])
        #input()
        #тут проверяем есть ли вообще данные о листах
        if P is not None:
            Id_hel=pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_id"]#id спиралей внутри файла в список
            #print(Id_hel)
            #Beg_str = pdbx_file [PDBid, "struct_conf"]["beg_auth_seq_id"]#начала спиралей а.о.
            Beg_str = pdbx_file [PDBid, "struct_conf"]["beg_label_seq_id"]#начала спиралей а.о.
            if isinstance(Beg_str, str):
                Beg_str=np.array([int(Beg_str)]) 
            else:
                Beg_str = Beg_str.astype("int")
            #print(Beg_str)
            End_str = pdbx_file [PDBid, "struct_conf"]["end_label_seq_id"]#концы спиралей а.о. ЕСЛИ КАК В БД брать нумерацию
            #End_str = pdbx_file [PDBid, "struct_conf"]["end_auth_seq_id"]#концы спиралей а.о.
            if isinstance(End_str, str):
                End_str=np.array([int(End_str)])
            else:
                End_str = End_str.astype("int")
            Class_hel = pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_class"]#классы спирали
            Length_hel = pdbx_file [PDBid, "struct_conf"]["pdbx_PDB_helix_length"]#длины спирали
            #Chain_hel = pdbx_file [PDBid, "struct_conf"]["beg_label_asym_id"]#цепи белка по буквам, каким принадлежат эти спирали
            Chain_hel = pdbx_file [PDBid, "struct_conf"]["beg_auth_asym_id"]#цепи белка по буквам, каким принадлежат эти спирали
            #print(Chain_hel)
            #ещё одна проверка если вдруг будет тупняк
            if Id_hel is not None:
                #сформируем локальный датафрейм альфа-спиралей, если эти альфа-спирали есть
                Helixes = pd.DataFrame({'Id_hel' : Id_hel,'Beg_str' : Beg_str, 'End_str': End_str, 'Class_hel': Class_hel, 'Length_hel': Length_hel, 'Chain_hel': Chain_hel})
                #print('Helixes\n', Helixes)
                #print('type', type(Helixes))
                #Удалим ненужные нам колонки (по названию цепи смотрим)
                Helixes = Helixes[Helixes.Class_hel == "5"]#тип спирали == альфа
                Helixes = Helixes[Helixes.Chain_hel == chain_tut]#цепь нужная по Podborka
                Helixes = Helixes[Helixes.Beg_str >= start_domain] #входит в домен
                Helixes = Helixes[Helixes.End_str <= stop_domain] #входит в домен
                Helixes.index = np.arange(len(Helixes))#обновляем индексы (чтобы стали от 0 до ..., а не как были в Helixes до того как удалим ненужные нам цепи)
                #print(" Helixes.index",  Helixes.index)
                #print("Helix которые подходят нам\n", Helixes)
                Podborka.loc[i, "Count_ThreeTenH"] = Helixes.shape[0]
                #если пусто по листам (их нет нужных), то тоже будем писать NaN или нули. тут повторная штука такая т.к. листов может не быть изначально (выше) или их не будет лишь в нужных нам цепях белка (тут)
                isempty = Helixes.empty#если нет 310-спиралей, то мы можем выбрать как заполнить данные
                if isempty:
                    print("В белке ", PDBid, "3_10-спиралей не обнаружено")
                    Res=[0]*100
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "ThreeTen-helixes"] = vse100
                    #print("Podborka", Podborka)
                else:
                    Res=[]
                    Res=list_struc_maker(Helixes, l, start_domain)#Получаем 100% данные структур
                    #print("Res", Res)
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "ThreeTen-helixes"] = vse100
                    #print("Podborka", Podborka)
            else:
                print("Какие-то проблемы с Id_hel")
        else:
            print("В белке ", PDBid, "3_10-спиралей не обнаружено")
            Podborka.loc[i, "Count_ThreeTenH"] = 0
            Res=[0]*100
            vse100 = (";".join(map(str,Res)))
            Podborka.loc[i, "ThreeTen-helixes"] = vse100
            #print("Podborka", Podborka)
    print('--------------------')        

def Beta_sheet(Podborka):
    g=0
    for i, row in Podborka.iterrows():
        #input()
        #print(f"Index: {i}")
        #print(f"{row}\n")
        PDBid = Podborka.iloc[i, 1]#выясняем актуальный PDBid 
        l = Podborka.iloc[i, 5]#выясняем актуальную длину
        chain_tut = Podborka.iloc[i, 2]#выясняем актуальную цепь
        start_domain = int(Podborka.iloc[i, 3])#выясняем актуальную координату начала домена/белка
        stop_domain = int(Podborka.iloc[i, 4])#выясняем актуальную координату конца домена/белка
        print("Данные: ", PDBid, l, chain_tut, start_domain, stop_domain, "g=", g)
        g=g+1
        #читаем PDBx из инета
        pdbx_file = pdbx.PDBxFile.read(rcsb.fetch(PDBid, "pdbx"))
        #Получаем данные из файла вообще есть они там или нет
        P=pdbx_file [PDBid, "struct_sheet_range"]#инфа о листах
        #print(pdbx_file["struct_sheet_order"])
        #input()
        #тут проверяем есть ли вообще данные о листах
        if P is not None:
            Id_sht=pdbx_file [PDBid, "struct_sheet_range"]["sheet_id"]#id листа
            #print(Id_sht)
            Beg_str = pdbx_file [PDBid, "struct_sheet_range"]["beg_label_seq_id"]#начало листа а.о.
            #Beg_str = pdbx_file [PDBid, "struct_sheet_range"]["beg_auth_seq_id"]#начало листа а.о.
            if isinstance(Beg_str, str):
                Beg_str=np.array([int(Beg_str)]) 
            else:
                Beg_str = Beg_str.astype("int")
            End_str = pdbx_file [PDBid, "struct_sheet_range"]["end_label_seq_id"]#конец листа а.о.
            #End_str = pdbx_file [PDBid, "struct_sheet_range"]["end_auth_seq_id"]#конец листа а.о.
            if isinstance(End_str, str):
                End_str=np.array([int(End_str)])
            else:
                End_str = End_str.astype("int")
            Chain_sht = pdbx_file [PDBid, "struct_sheet_range"]["beg_auth_asym_id"]#цепь https://www.biotite-python.org/examples/gallery/structure/sheet_arrangement.html
            #print("chain_sht", Chain_sht)
            #input()
            #ещё одна проверка если вдруг будет тупняк
            if Id_sht is not None:
                #сформируем локальный датафрейм альфа-спиралей, если эти альфа-спирали есть
                Sheeets = pd.DataFrame({'Id_sht' : Id_sht,'Beg_str' : Beg_str, 'End_str': End_str, 'Chain_sht': Chain_sht})
                #print('Sheeets\n', Sheeets)
                #print('type', type(Sheeets))
                #Удалим ненужные нам колонки (по названию цепи смотрим)
                Sheeets = Sheeets[Sheeets.Chain_sht == chain_tut]#цепь нужная по Podborka
                Sheeets = Sheeets[Sheeets.Beg_str >= start_domain] #входит в домен
                Sheeets = Sheeets[Sheeets.End_str <= stop_domain] #входит в домен
                Sheeets.index = np.arange(len(Sheeets))#обновляем индексы (чтобы стали от 0 до ..., а не как были в Helixes до того как удалим ненужные нам цепи)
                #print(" Sheeets.index",  Sheeets.index)
                #print("Sheeets которые подходят нам\n", Sheeets)
                Podborka.loc[i, "Count_BetaS"] = Sheeets.shape[0]
                #если пусто по листам (их нет нужных), то тоже будем писать NaN или нули. тут повторная штука такая т.к. листов может не быть изначально (выше) или их не будет лишь в нужных нам цепях белка (тут)
                isempty = Sheeets.empty#если нет 310-спиралей, то мы можем выбрать как заполнить данные
                if isempty:
                    print("В белке ", PDBid, "бета-листов не обнаружено")
                    Res=[0]*100
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "Beta-sheets"] = vse100
                    #print("Podborka", Podborka)
                else:
                    Res=[]
                    Res=list_struc_maker(Sheeets, l, start_domain)#Получаем 100% данные структур
                    #print("Res", Res)
                    vse100 = (";".join(map(str,Res)))
                    Podborka.loc[i, "Beta-sheets"] = vse100
                    #print("Podborka", Podborka)
            else:
                print("Какие-то проблемы с Id_sht")
        else:
            print("В белке ", PDBid, "бета-листов не обнаружено")
            Podborka.loc[i, "Count_BetaS"] = 0
            Res=[0]*100
            vse100 = (";".join(map(str,Res)))
            Podborka.loc[i, "Beta-sheets"] = vse100
            #print("Podborka", Podborka)
    print('--------------------')        


def Irregular(Podborka):
    for i, row in Podborka.iterrows():
        PDBid = Podborka.iloc[i, 1]#выясняем актуальный PDBid просто для отслеживания
        #l = Podborka.iloc[i, 5]#выясняем актуальную длину
        #print("l=", l)
        #print("Podborka.iloc[i, 6]", Podborka.iloc[i, 6])
        list_alpha = Podborka.iloc[i, 6].split(";")#делаем список с данными об альфа
        list_beta = Podborka.iloc[i, 7].split(";")#делаем список с данными о бета-листах
        list_threeten = Podborka.iloc[i, 8].split(";")#делаем список с данными о 310-спиралях
        print("Данные: ", PDBid)
        #print("list_alpha=",list_alpha)
        proba = [0]*100#пустой для наших нерегулярных структур        
        for index in range(len(proba)):
            #print("index=", index, ", list_alpha[index]=", list_alpha[index])
            #print(type(l),"   ", type(float(list_alpha[index])))
            proba[index]=1-float(list_alpha[index])-float(list_beta[index])-float(list_threeten[index])
            proba[index] = round(proba[index], 2)
            if proba[index] < 0:
                print("значение в нерегулярных стурктурах <0!")
                print(PDBid, "index=", index, "proba=", proba)
                #input()
            #print("proba=", proba)
            #vse100 = ';'.join(str(proba))
            vse100 = (";".join(map(str,proba)))
            #print("vse100=", vse100)
            #vse100 = (";", join(map(str, proba)))
            Podborka.loc[i, "Irregular"] = vse100   
    print('--------------------') 
    
            
def list_struc_maker(Helixes, l, start_domain):
    #Считываем нужные альфа-спирали в список 
    list_hel_original=[0] * l#список, где будем выставлять наличие альфа-спиралей
    #заполняем его (это тот, что в аминокислотных остатках)
    for i in range(0,len(Helixes)):
        start = int(Helixes.iloc[i]["Beg_str"])
        stop = int(Helixes.iloc[i]["End_str"])
        #print("СТАРТ=", start, " СТОП=", stop)
        #ставим относительно начала и конца домена с учётом, что в списке начало с 0, а не 1 и учёт домена
        start = start-start_domain
        stop = stop - start_domain
        #print("start=", start, " stop=", stop)
        #вносим в нашу цепь данные
        for c in range (start, stop+1):
            list_hel_original[c]=1
            #print("list_hel_original", list_hel_original[c])
    #print("Итого по цепи")
    #for i in range(0, len(list_hel_original)):
        #print("list_hel_original[",i,"]=", list_hel_original[i])
    Itog = translater(list_hel_original, l)
    #print("itog",Itog)
    return Itog
    print('--------------------')

def translater(list_hel_original, l):
    #делаем список из 100 ячеек (наши координаты, т.е. %)
    proba = [0]*100
    T=l/100#Сколько приходится аминокислот на 1 ячейку
    print("T=", T)
    #if T>1:
    if T!=1:
        k=1
        t=T
        #print("t=", t)
        i=1
        j=1
        while i<l and j<100:
            a=min(k, t)
            proba[j]=round(proba[j]+a*list_hel_original[i],2)
            k=round(k-a,2)
            t=round(t-a,2)
            #print("a=",a," значение[",i,"]=", list_hel_original[i]," proba[", j,"]=", proba[j], " k=",k," t=",t, " i=", i, " j=",j)
            if k==0:
                i=i+1
                k=1
            if t==0:
                j=j+1
                t=T
            #print("k=",k, " t=", t)
        proba = [round(x/T, 2) for x in proba]
        #print("proba\n", proba)
    if T==1:
        proba=list_hel_original#просто копируем
    #if T<1:  
    #print("proba\n", proba)
    #print("proba\n", proba)
    return proba
    print('--------------------')



print('Добро пожаловать!')
start_time = time.perf_counter() #счётчик времени выполнения проги
        
df = Start()
Change_df (df)
Hist_maker(df)


print("Выберите какие структуры Вы хотели бы просмотреть: 1-альфа-спирали, 2-спирали 3_10, 3-бета-листы, 0-все приведённые и нерегулярные структуры.")
v = input()
print("Вы ввели: ", v)
if (v == "1"):
    print("Начинаю просматривать альфа-спирали")
    Alpha_helix(df)
else:
    if (v == "2"):
        print("Начинаю просматривать спирали 3-10") 
        ThreeTen_helix(df)
    else:
        if (v == "3"):
            print("Начинаю просматривать бета-листы")
            Beta_sheet(df)
        else: 
            if (v == "0"):
                print("Начинаю просматривать альфа-спирали")
                Alpha_helix(df)
                print("Начинаю просматривать спирали 3-10") 
                ThreeTen_helix(df)
                print("Начинаю просматривать бета-листы")
                Beta_sheet(df)
                print("Начинаю просматривать нерегулярные структуры")
                Irregular(df)
            else: print("Вы ввели неверное значение v. Попробуйте ещё раз")

#input()
#print(df.head)

df.to_excel("Result.xlsx")         
                   
print('Время работы скрипта: ',time.perf_counter()-start_time, 'сек')
#Время работы скрипта:  1739.8240654 сек // ~29минут